In [1]:
# Import 需要的套件
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc

import matplotlib.pyplot as plt
%matplotlib inline

# Models Packages
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

In [2]:
# Part 1 讀取檔案
dir_data = './ml100marathon/'
f_app_train = os.path.join(dir_data, 'train_data.csv')
f_app_test = os.path.join(dir_data, 'test_features.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

# 保留train的筆數長度，供後續使用
len_train = len(app_train)

print('train shape:', app_train.shape)
print('test shape:', app_test.shape)

train shape: (113, 22)
test shape: (33, 21)


In [3]:
# 合併起來(方便一起做資料加工)
all_data = pd.concat([app_train, app_test], ignore_index=True)
print('all data shape:', all_data.shape)
all_data.head(5)

all data shape: (146, 22)


,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,name,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,...,RICE KENNETH D,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,...,SKILLING JEFFREY K,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,...,SHELBY REX,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,...,KOPPER MICHAEL J,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,...,CALGER CHRISTOPHER F,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [4]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 22 columns):
bonus                        82 non-null float64
deferral_payments            39 non-null float64
deferred_income              49 non-null float64
director_fees                17 non-null float64
email_address                111 non-null object
exercised_stock_options      102 non-null float64
expenses                     95 non-null float64
from_messages                86 non-null float64
from_poi_to_this_person      86 non-null float64
from_this_person_to_poi      86 non-null float64
loan_advances                4 non-null float64
long_term_incentive          66 non-null float64
name                         146 non-null object
other                        93 non-null float64
poi                          113 non-null object
restricted_stock             110 non-null float64
restricted_stock_deferred    18 non-null float64
salary                       95 non-null float64
shared_recei

In [5]:
all_data.describe()

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,8.200000e+01,3.900000e+01,4.900000e+01,1.700000e+01,1.020000e+02,9.500000e+01,86.000000,86.000000,86.000000,4.000000e+00,6.600000e+01,9.300000e+01,1.100000e+02,1.800000e+01,9.500000e+01,86.000000,86.000000,1.250000e+02,1.260000e+02
mean,2.374235e+06,1.642674e+06,-1.140475e+06,1.668049e+05,5.987054e+06,1.087289e+05,608.790698,64.895349,41.232558,4.196250e+07,1.470361e+06,9.190650e+05,2.321741e+06,1.664106e+05,5.621943e+05,1176.465116,2073.860465,5.081526e+06,6.773957e+06
std,1.071333e+07,5.161930e+06,4.025406e+06,3.198914e+05,3.106201e+07,5.335348e+05,1841.033949,86.979244,100.073111,4.708321e+07,5.942759e+06,4.589253e+06,1.251828e+07,4.201494e+06,2.716369e+06,1178.317641,2582.700981,2.906172e+07,3.895777e+07
min,7.000000e+04,-1.025000e+05,-2.799289e+07,3.285000e+03,3.285000e+03,1.480000e+02,12.000000,0.000000,0.000000,4.000000e+05,6.922300e+04,2.000000e+00,-2.604490e+06,-7.576788e+06,4.770000e+02,2.000000,57.000000,1.480000e+02,-4.409300e+04
25%,4.312500e+05,8.157300e+04,-6.948620e+05,9.878400e+04,5.278862e+05,2.261400e+04,22.750000,10.000000,1.000000,1.600000e+06,2.812500e+05,1.215000e+03,2.540180e+05,-3.896218e+05,2.118160e+05,249.750000,541.250000,3.944750e+05,4.945102e+05
50%,7.693750e+05,2.274490e+05,-1.597920e+05,1.085790e+05,1.310814e+06,4.695000e+04,41.000000,35.000000,8.000000,4.176250e+07,4.420350e+05,5.238200e+04,4.517400e+05,-1.469750e+05,2.599960e+05,740.500000,1211.000000,1.101393e+06,1.102872e+06
75%,1.200000e+06,1.002672e+06,-3.834600e+04,1.137840e+05,2.547724e+06,7.995250e+04,145.500000,72.250000,24.750000,8.212500e+07,9.386720e+05,3.620960e+05,1.002370e+06,-7.500975e+04,3.121170e+05,1888.250000,2634.750000,2.093263e+06,2.949847e+06
max,9.734362e+07,3.208340e+07,-8.330000e+02,1.398517e+06,3.117640e+08,5.235198e+06,14368.000000,528.000000,609.000000,8.392500e+07,4.852193e+07,4.266759e+07,1.303223e+08,1.545629e+07,2.670423e+07,5521.000000,15149.000000,3.098866e+08,4.345095e+08


In [6]:
# Part 2 缺失值處理
all_data.drop(['email_address'], axis=1, inplace=True)
#all_data.drop(['poi'], axis=1, inplace=True)
all_data = all_data.fillna(-999)
all_data.head(5)

,bonus,deferral_payments,deferred_income,director_fees,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,...,name,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,-999.0,-3504386.0,-999.0,19794175.0,46950.0,18.0,42.0,4.0,-999.0,...,RICE KENNETH D,174839.0,True,2748364.0,-999.0,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,-999.0,-999.0,-999.0,19250000.0,29336.0,108.0,88.0,30.0,-999.0,...,SKILLING JEFFREY K,22122.0,True,6843672.0,-999.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,-999.0,-4167.0,-999.0,1624396.0,22884.0,39.0,13.0,14.0,-999.0,...,SHELBY REX,1573324.0,True,869220.0,-999.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,-999.0,-999.0,-999.0,-999.0,118134.0,-999.0,-999.0,-999.0,-999.0,...,KOPPER MICHAEL J,907502.0,True,985032.0,-999.0,224305.0,-999.0,-999.0,2652612.0,985032.0
4,1250000.0,-999.0,-262500.0,-999.0,-999.0,35818.0,144.0,199.0,25.0,-999.0,...,CALGER CHRISTOPHER F,486.0,True,126027.0,-999.0,240189.0,2188.0,2598.0,1639297.0,126027.0


In [7]:
# Part 3 特徵工程
# LabelEncoder
le = preprocessing.LabelEncoder()

# 字串欄位轉數值
le.fit(all_data['name'].astype(str))
all_data['name'] = le.transform(all_data['name'].astype(str))

le.fit(all_data['poi'].astype(str))
all_data['poi'] = le.transform(all_data['poi'].astype(str))

In [8]:
# Part 4
# 把整理好的資料黏回去(利用最早紀錄的長度當索引)

# all_data裡面第一筆到train長度 為訓練資料
train_df = all_data[:len_train]

# all_data裡面從train長度起始到最後一筆 為測試資料
test_df = all_data[len_train:]

print('saving training and testing set..')
train_df.to_csv('ml100marathon/train_data_prep.csv',index = False, header=True, encoding='utf-8')
test_df.to_csv('ml100marathon/test_data_prep.csv',index = False, header=True, encoding='utf-8')

saving training and testing set..


In [9]:
# Part 5 測試驗證
#import lightgbm as lgbm
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

train_df = pd.read_csv('ml100marathon/train_data_prep.csv')
test_df = pd.read_csv('ml100marathon/test_data_prep.csv')

print(train_df.shape)
print(test_df.shape)

# 從原始test資料抽出(pop)並保留裡面的乘客id (後續mapping)
test_passenger_ids = test_df.pop('name')

# 丟掉train裡面的乘客id (不需要給模型當特徵來訓練)
train_df.drop(['name'], axis=1, inplace=True)
    
# 從原始train資料抽出(pop)並存放所有答案
y = train_df.pop('poi')

# 拆出訓練與驗證資料
X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=0.2, random_state=42)

(113, 21)
(33, 21)


In [13]:
# 建立模型
# logreg = linear_model.LogisticRegression(solver='liblinear',random_state=0)

# # 訓練模型
# logreg.fit (X_train, y_train)

# # 預測測試集
# y_pred = logreg.predict(X_test)

# print(f"Accuracy = {accuracy_score(y_test, y_pred):.3f}")

# 建立一個線性回歸模型
lasso = linear_model.Lasso(alpha=1.0)

# 將訓練資料丟進去模型訓練
lasso.fit(X_train, y_train)

# 將測試資料丟進模型得到預測結果
y_pred = lasso.predict(X_test)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [14]:
submission = pd.DataFrame(y_pred)
print(submission.shape)
submission.columns = ['poi']
submission['name'] = np.arange(1,submission.shape[0]+1)
submission = submission[['name', 'poi']]
submission

(23, 1)


,name,poi
0,1,1.213814
1,2,1.068888
2,3,1.020921
3,4,1.249291
4,5,1.199846
5,6,1.210240
6,7,0.967507
7,8,1.081228
8,9,1.197975
9,10,1.150497


In [ ]:
# dataframe to CSV file
submission.to_csv('submission.csv', index=False)